<img src="https://raw.githubusercontent.com/OpenEnergyPlatform/academy/develop/docs/data/img/OEP_logo_2_no_text.svg" alt="OpenEnergy Platform" height="75" width="75" align="left"/>

# API tutorial 4 - Process query result data and save to file

Repository: https://github.com/openego/oedialect

Please report bugs and improvements here: https://github.com/OpenEnergyPlatform/examples/issues <br>
How to get started with Jupyter Notebooks can be found here: https://realpython.com/jupyter-notebook-introduction/ <br>
Please ensure you have read the Terms of use here: https://openenergy-platform.org/legal/tou/

In [1]:
__copyright__ = "Zentrum für nachhaltige Energiesysteme Flensburg"
__license__   = "GNU Affero General Public License Version 3 (AGPL-3.0)"
__url__       = "https://github.com/openego/data_processing/blob/master/LICENSE"
__author__    = "wolfbunke"

## Introduction

<br>
<div class="alert alert-block alert-danger">
This is an important information!
</div>
<div class="alert alert-block alert-info">
This is an information!
</div>
<div class="alert alert-block alert-success">
This is your task!
</div>

This tutorial gives you an overview of the [**OpenEnergy Platform**](https://openenergy-platform.org/) and how you can work with the **REST-full-HTTP** API in Python. <br>
The full API documentaion can be found on [ReadtheDocs.io](http://oep-data-interface.readthedocs.io/en/latest/api/how_to.html#authenticate "OpenEnergyPlatform’s documentation").


## Part IV - How to work with the OpenEnergy Platform (OEP)

0 Setup token <br>
1 Select data <br>
2 Make a pandas dataframe <br>
3 Make calculations <br>
4 Save results as csv and excel files <br>


In [2]:
# pip install requests pandas xlsxwriter

import requests
import pandas as pd
import xlsxwriter

# Part  IV

## 1. Select data

In [3]:
# select powerplant data
schema = 'supply'
table = 'ego_dp_conv_powerplant'
where = 'version=v0.2.10'
oep_url = 'https://openenergy-platform.org'
conv_powerplants = requests.get(oep_url+'/api/v0/schema/'+schema+'/tables/'+table+'/rows/?where='+where, )
conv_powerplants.status_code

200

<div class="alert alert-block alert-info">
<b>Response [200]</b> succesfully selected data! <br>
<b>Response [404]</b> table doesn't exist!
</div>

## 2. Make a pandas dataframe

In [4]:
df_pp = pd.DataFrame(conv_powerplants.json())

## 3. Make calculations

<div class="alert alert-block alert-success">
Get an overview of your DataFrame:
</div>

In [5]:
df_pp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 879 entries, 0 to 878
Data columns (total 42 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   version                879 non-null    object 
 1   id                     879 non-null    int64  
 2   bnetza_id              879 non-null    object 
 3   company                876 non-null    object 
 4   name                   845 non-null    object 
 5   postcode               879 non-null    object 
 6   city                   879 non-null    object 
 7   street                 476 non-null    object 
 8   state                  879 non-null    object 
 9   block                  638 non-null    object 
 10  commissioned_original  878 non-null    object 
 11  commissioned           877 non-null    float64
 12  retrofit               76 non-null     float64
 13  shutdown               54 non-null     float64
 14  status                 879 non-null    object 
 15  fuel  

<div class="alert alert-block alert-success">
Sum the installed Capacity by fuels and add the unit MW in a new column.
</div>

In [6]:
results = df_pp[['capacity','fuel']].groupby('fuel').sum()
results['units'] = 'MW'
results

,capacity,units
fuel,,
biomass,1195.5200,MW
coal,31468.3000,MW
gas,27275.9925,MW
gas_mine,75.4000,MW
lignite,22931.0000,MW
oil,4466.6000,MW
other_non_renewable,2592.5000,MW
pumped_storage,9275.2000,MW
reservoir,1382.5000,MW


<div class="alert alert-block alert-success">
Create a csv file from the data frame with a suitable name and configure the representation of the different data types. 
</div>

In [7]:
# Write DataFrame as csv with desired column seperator and representation of the datatypes
results.to_csv(
    'output_example_conventional_powerplants_germany.csv',
    sep=',',
    float_format='%.3f',
    decimal='.',
    date_format='%Y-%m-%d',
    encoding='utf-8'
)


<div class="alert alert-block alert-success">
Create a .xlsx file with a suitable name and two sheets. The sheets must contain the data from 1. result and 2. df_pp.
</div>

In [8]:
# Write the results as xlsx file
writer = pd.ExcelWriter('output_example_onventional_powerplants_germany.xlsx', engine='xlsxwriter')

# write results of installed Capacity by fuels
results.to_excel(writer, index=False, sheet_name='Installed Capacities by fuel')

# write orgininal data in second sheet
df_pp.to_excel(writer, index=False, sheet_name='Conventional Powerplants')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

AttributeError: 'XlsxWriter' object has no attribute 'save'

<div class="alert alert-block alert-info">
The file can be found in the <b>/api</b> folder.
</div>